In [1]:
# loading pretrained inception V3 model for generating deep dream images

import keras
from keras.applications import inception_v3
from keras import backend as K

# We will not be training our model,
# so we use this command to disable all training-specific operations
K.set_learning_phase(0)

# Build the InceptionV3 network.
# The model will be loaded with pre-trained ImageNet weights.
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

Using TensorFlow backend.
W0716 09:21:14.365431 140347094234880 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0716 09:21:14.367186 140347094234880 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:442: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0716 09:21:14.369290 140347094234880 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3543: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0716 09:21:14.456218 140347094234880 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backe

87801856/87910968 [============================>.] - ETA: 0s

W0716 09:21:32.329392 140347094234880 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0716 09:21:32.330915 140347094234880 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:158: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [2]:
# setting the deepdream config, i.e. what layers should contribute to the final image
# this is a somewhat arbitraty selection but can be changed and experimented with

# Dict mapping layer names to a coefficient quantifying how much the layer's activation
# will contribute to the loss we will seek to maximize.
# Note that these are layer names as they appear in the built-in InceptionV3 application.
# You can list all layer names using `model.summary()`.
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [3]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [8]:
# we seek to optimize a loss defined by the weighted sum of L2 norm
# of the activations of the layers defined above

# Get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Define the loss.
loss = K.variable(0.) #instantiating a variable with value 0
for layer_name in layer_contributions:
    # Add the L2 norm of the features of a layer to the loss.
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

W0716 09:40:47.696925 140347094234880 variables.py:2569] Variable += will be deprecated. Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.


In [7]:
layer_dict

{'activation_1': <keras.layers.core.Activation at 0x7fa4b7eab668>,
 'activation_10': <keras.layers.core.Activation at 0x7fa4b7055e10>,
 'activation_11': <keras.layers.core.Activation at 0x7fa4b701d9b0>,
 'activation_12': <keras.layers.core.Activation at 0x7fa4b6f806a0>,
 'activation_13': <keras.layers.core.Activation at 0x7fa4b6f68dd8>,
 'activation_14': <keras.layers.core.Activation at 0x7fa4b6ec9780>,
 'activation_15': <keras.layers.core.Activation at 0x7fa4b6eae9b0>,
 'activation_16': <keras.layers.core.Activation at 0x7fa4b6df7da0>,
 'activation_17': <keras.layers.core.Activation at 0x7fa4b6ddd940>,
 'activation_18': <keras.layers.core.Activation at 0x7fa4b6d3eef0>,
 'activation_19': <keras.layers.core.Activation at 0x7fa4b6d25550>,
 'activation_2': <keras.layers.core.Activation at 0x7fa4b7356c18>,
 'activation_20': <keras.layers.core.Activation at 0x7fa4b6c869e8>,
 'activation_21': <keras.layers.core.Activation at 0x7fa4b6c69390>,
 'activation_22': <keras.layers.core.Activation at